In [1]:
!pip install datasets


In [2]:
from datasets import load_dataset
dataset = load_dataset("trec")
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'coarse_label', 'fine_label'],
        num_rows: 5452
    })
    test: Dataset({
        features: ['text', 'coarse_label', 'fine_label'],
        num_rows: 500
    })
})

In [3]:
dataset['train']

Dataset({
    features: ['text', 'coarse_label', 'fine_label'],
    num_rows: 5452
})

In [4]:
trainx=dataset['train']['text']
trainy=dataset['train']['coarse_label']
testx=dataset['test']['text']
testy=dataset['test']['coarse_label']
def makey(fil2):
  a=[]
  i=0
  for b in fil2:
      a.append([max(1-b,0),max(-2*((b-1)**2)+1,0),max(-2*((b-2)**2)+1,0),max(-2*((b-3)**2)+1,0),max(-2*((b-4)**2)+1,0),max(-2*((b-5)**2)+1,0)])
  return a
trainy=makey(trainy)
testy=makey(testy)


In [39]:
from zipfile import ZipFile
import tensorflow.keras as K
import tensorflow
from tensorflow.keras import layers
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding
from sklearn.metrics import confusion_matrix

In [6]:
from tensorflow.keras.layers import TextVectorization
vectorizer = TextVectorization(standardize='lower_and_strip_punctuation',output_sequence_length=33)
text_ds = tensorflow.data.Dataset.from_tensor_slices(trainx+testx).batch(128)
vectorizer.adapt(text_ds)
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

trainx = vectorizer(trainx)
testx = vectorizer(testx)


In [7]:
def bow(dato):
  bow_data=[]
  for a in dato:
    b= [0 for i in range(len(voc))]
    for c in a:
      b[int(c)]=b[int(c)]+1
    bow_data.append(b)
  return bow_data

bowtrainx=bow(trainx)
print(bowtrainx[0])



[24, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [8]:
bowtestx=bow(testx)

In [9]:
trainx= np.array(trainx)
testx= np.array(testx)
trainy= np.array(trainy)
testy= np.array(testy)

In [42]:


simple_model=K.Sequential([
    tensorflow.keras.Input(shape=(33,),batch_size=10),
    Embedding(len(voc)+2,300,trainable=True),
    layers.Flatten(),
    layers.Dense(6, activation="softmax",kernel_regularizer=K.regularizers.l2(0.00001))
])
simple_model.compile(optimizer=tensorflow.keras.optimizers.Adadelta(learning_rate=1),
              loss='binary_crossentropy',
              metrics=['accuracy'])
epochs=30
batch_size=10
callback = K.callbacks.EarlyStopping(monitor='loss', patience=3,min_delta=-0.01)
simple_model.fit(trainx, trainy,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                callbacks=[callback])
score = simple_model.evaluate(testx, testy, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

y_pred = simple_model.predict(testx)
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(testy, axis=1)
cm = confusion_matrix(y_test, y_pred)
print(cm)


Epoch 1/30
546/546 [==============================] - 22s 40ms/step - loss: 0.3322 - accuracy: 0.5211
Epoch 2/30
546/546 [==============================] - 22s 40ms/step - loss: 0.2417 - accuracy: 0.6820
Epoch 3/30
546/546 [==============================] - 22s 41ms/step - loss: 0.2009 - accuracy: 0.7564
Epoch 4/30
546/546 [==============================] - 22s 41ms/step - loss: 0.1715 - accuracy: 0.8109
Epoch 5/30
546/546 [==============================] - 24s 43ms/step - loss: 0.1476 - accuracy: 0.8494
Epoch 6/30
546/546 [==============================] - 21s 38ms/step - loss: 0.1272 - accuracy: 0.8813
Epoch 7/30
546/546 [==============================] - 22s 40ms/step - loss: 0.1100 - accuracy: 0.9098
Epoch 8/30
546/546 [==============================] - 20s 38ms/step - loss: 0.0955 - accuracy: 0.9275
Epoch 9/30
546/546 [==============================] - 21s 39ms/step - loss: 0.0832 - accuracy: 0.9402
Epoch 10/30
546/546 [==============================] - 21s 38ms/step - loss: 0.072

In [48]:
simple_model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (10, 33, 300)             2671200   
                                                                 
 flatten_13 (Flatten)        (10, 9900)                0         
                                                                 
 dense_17 (Dense)            (10, 6)                   59406     
                                                                 
Total params: 2,730,606
Trainable params: 2,730,606
Non-trainable params: 0
_________________________________________________________________


In [38]:
num=[0,0,0,0,0,0]
for a in dataset['train']['coarse_label']:
  num[int(a)]=num[int(a)]+1
print(num)
num=[0,0,0,0,0,0]
for a in dataset['test']['coarse_label']:
  num[int(a)]=num[int(a)]+1
print(num)

[86, 1250, 1162, 1223, 835, 896]
[9, 94, 138, 65, 81, 113]


In [43]:

bowtestx= np.array(bowtestx)
bowtrainx= np.array(bowtrainx)

model1=K.Sequential([
    tensorflow.keras.Input(shape=(8902,),batch_size=10),
    layers.Dense(6, activation="softmax",kernel_regularizer=K.regularizers.l2(0.00001))
])
model1.compile(optimizer=tensorflow.keras.optimizers.Adadelta(learning_rate=1),
              loss='binary_crossentropy',
              metrics=['accuracy'])
epochs=30
batch_size=10
callback = K.callbacks.EarlyStopping(monitor='loss', patience=3,min_delta=-0.01)
model1.fit(bowtrainx, trainy,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                callbacks=[callback])
score = model1.evaluate(bowtestx, testy, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])



Epoch 1/30
546/546 [==============================] - 2s 3ms/step - loss: 0.4114 - accuracy: 0.4283
Epoch 2/30
546/546 [==============================] - 2s 3ms/step - loss: 0.3536 - accuracy: 0.5866
Epoch 3/30
546/546 [==============================] - 2s 3ms/step - loss: 0.3236 - accuracy: 0.6313
Epoch 4/30
546/546 [==============================] - 2s 3ms/step - loss: 0.3022 - accuracy: 0.6594
Epoch 5/30
546/546 [==============================] - 2s 4ms/step - loss: 0.2862 - accuracy: 0.6792
Epoch 6/30
546/546 [==============================] - 2s 4ms/step - loss: 0.2739 - accuracy: 0.6902
Epoch 7/30
546/546 [==============================] - 2s 3ms/step - loss: 0.2641 - accuracy: 0.6985
Epoch 8/30
546/546 [==============================] - 2s 3ms/step - loss: 0.2557 - accuracy: 0.7093
Epoch 9/30
546/546 [==============================] - 2s 3ms/step - loss: 0.2490 - accuracy: 0.7166
Epoch 10/30
546/546 [==============================] - 2s 3ms/step - loss: 0.2426 - accuracy: 0.7262

ValueError: ignored

In [47]:
model1.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (10, 6)                   53418     
                                                                 
Total params: 53,418
Trainable params: 53,418
Non-trainable params: 0
_________________________________________________________________


In [44]:
y_pred = model1.predict(bowtestx)
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(testy, axis=1)
cm = confusion_matrix(y_test, y_pred)
print(cm)

16/16 [==============================] - 0s 6ms/step
[[  5   0   4   0   0   0]
 [  0  60  17   8   9   0]
 [  0   2 135   0   0   1]
 [  0   8   1  55   1   0]
 [  0  11   9   0  61   0]
 [  0  13  27   0   7  66]]


In [12]:
with ZipFile("glove.6B.zip", 'r') as z:
    z.extract(
        "glove.6B.300d.txt")
z.close()

In [13]:

embeddings_index = {}
f=open('glove.6B.300d.txt')
for line in f:
    word,coefs= line.split(maxsplit=1)
    coefs = np.fromstring(coefs, "f", sep=" ")
    embeddings_index[word] = coefs
hits=0
misses=0
embedding_matrix = np.zeros((len(voc) + 2, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 8452 words (450 misses)


In [14]:
print(trainx[1])
print(testy[0])

[   3  940  680    2  133 1387 7335    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
[0 0 0 0 0 1]


In [15]:
max=0
for line in trainx:
  if len(line)>max:
    max=len(line)
print(max)
print(len(voc))

33
8902


In [45]:


model2 = K.Sequential(
    [
        Embedding(len(voc) + 2,300,embeddings_initializer=K.initializers.Constant(embedding_matrix),trainable=True),
        layers.Dropout(0.5),
        layers.Bidirectional(layers.LSTM(300,dropout=0.2,return_sequences=True,recurrent_regularizer=K.regularizers.l2(0.00001)), merge_mode="sum"),
        layers.Dropout(0.2),
        layers.Reshape((33,300,1)),
        layers.Conv2D(100,(3,3),activation="tanh",kernel_regularizer=K.regularizers.l2(0.00001)),
        layers.MaxPooling2D(),
        layers.Dropout(0.4),
        layers.Flatten(),
        layers.Dense(6, activation="softmax")
    ]
)
print(model2.summary())

model2.compile(optimizer=tensorflow.keras.optimizers.Adadelta(learning_rate=1),
              loss='binary_crossentropy',
              metrics=['accuracy'])
epochs=30
batch_size=10
callback = K.callbacks.EarlyStopping(monitor='loss', patience=3,min_delta=-0.01)
model2.fit(trainx, trainy,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                callbacks=[callback])
score = model2.evaluate(testx, testy, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

y_pred = model2.predict(testx)
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(testy, axis=1)
cm = confusion_matrix(y_test, y_pred)
print(cm)

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_16 (Embedding)    (None, None, 300)         2671200   
                                                                 
 dropout_21 (Dropout)        (None, None, 300)         0         
                                                                 
 bidirectional_7 (Bidirectio  (None, None, 300)        1442400   
 nal)                                                            
                                                                 
 dropout_22 (Dropout)        (None, None, 300)         0         
                                                                 
 reshape_7 (Reshape)         (None, 33, 300, 1)        0         
                                                                 
 conv2d_22 (Conv2D)          (None, 31, 298, 100)      1000      
                                                     

In [50]:
model3 = K.Sequential(
    [
        Embedding(len(voc) + 2,300,embeddings_initializer=K.initializers.Constant(embedding_matrix),trainable=True),
        layers.Dropout(0.5),
        layers.Bidirectional(layers.LSTM(150,dropout=0.2,return_sequences=True,recurrent_regularizer=K.regularizers.l2(0.00001)), merge_mode="sum"),
        layers.Dropout(0.2),
        layers.Reshape((33,150,1)),
        layers.Conv2D(100,(3,3),activation="tanh",kernel_regularizer=K.regularizers.l2(0.00001)),
        layers.MaxPooling2D(),
        layers.Conv2D(75,(3,3),activation="tanh",kernel_regularizer=K.regularizers.l2(0.00001)),
        layers.MaxPooling2D(),
        layers.Conv2D(50,(3,3),activation="tanh",kernel_regularizer=K.regularizers.l2(0.00001)),
        layers.Dropout(0.4),
        layers.Flatten(),
        layers.Dense(6, activation="softmax")
    ]
)
print(model3.summary())

model3.compile(optimizer=tensorflow.keras.optimizers.Adadelta(learning_rate=1),
              loss='binary_crossentropy',
              metrics=['accuracy'])
epochs=30
batch_size=10
callback = K.callbacks.EarlyStopping(monitor='loss', patience=3,min_delta=-0.01)
model3.fit(trainx, trainy,
                epochs=epochs,
                batch_size=batch_size,
                shuffle=True,
                callbacks=[callback])
score = model3.evaluate(testx, testy, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_19 (Embedding)    (None, None, 300)         2671200   
                                                                 
 dropout_30 (Dropout)        (None, None, 300)         0         
                                                                 
 bidirectional_10 (Bidirecti  (None, None, 150)        541200    
 onal)                                                           
                                                                 
 dropout_31 (Dropout)        (None, None, 150)         0         
                                                                 
 reshape_10 (Reshape)        (None, 33, 150, 1)        0         
                                                                 
 conv2d_29 (Conv2D)          (None, 31, 148, 100)      1000      
                                                     

In [52]:
y_pred = model3.predict(testx)
y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(testy, axis=1)
cm = confusion_matrix(y_test, y_pred)
print(cm)

16/16 [==============================] - 4s 260ms/step
[[  7   0   2   0   0   0]
 [  0  81   4   2   5   2]
 [  0   1 135   0   0   2]
 [  0   0   1  62   2   0]
 [  0   1   0   0  79   1]
 [  0   0   4   0   3 106]]
